In [ ]:
# Imported here in this order to enforce logging of Tensorflow
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf

In [ ]:
import pandas as pd
from anaconda.enterprise.server.common.sdk import load_ae5_user_secrets
from dotenv import load_dotenv

# load defined environmental variables
load_ae5_user_secrets()
load_dotenv(dotenv_path="../env/env.dev")

In [ ]:
from tensorflow.python.trackable.autotrackable import AutoTrackable
from typing import Union, Any
import mlflow


def load_model(model_name: str, model_version: int) -> Union[None | list[Any | None] | AutoTrackable]:
    model_uri: str = f"models:/{model_name}/{model_version}"
    return mlflow.tensorflow.load_model(model_uri=model_uri)

In [ ]:
import numpy as np
import cv2


def load_image(image_path: str) -> np.ndarray:
    return cv2.cvtColor(cv2.imread(image_path, cv2.IMREAD_REDUCED_COLOR_8), cv2.COLOR_BGR2RGB)

In [ ]:
from pathlib import Path

# import tensorflow as tf

IMAGE_HEIGHT: int = 28
IMAGE_WIDTH: int = 28

# dimensions for resize for model input
dim = (IMAGE_WIDTH, IMAGE_HEIGHT)


def generate_model_input_single(file: Path) -> tf.Tensor:
    image_array: np.ndarray = load_image(image_path=file.resolve().as_posix())

    # resize for model input
    image_array_resized = cv2.resize(image_array, dim)

    # expand the dimensions of the input for a batch of 1
    model_input: tf.Tensor = tf.expand_dims(image_array_resized, 0)

    return model_input

In [ ]:
from keras import engine

# Load a model
model_name: str = "jburt.dev.mlflow.classification.workflow"
model_version: int = 3
model: engine.functional.Functional = load_model(model_name=model_name, model_version=model_version)

In [ ]:
from keras import engine
from anaconda.enterprise.server.contracts import BaseModel


class Predictor(BaseModel):
    model: engine.functional.Functional

    @staticmethod
    def _pre_process(input_image: Path) -> tf.Tensor:
        input_tensor: tf.Tensor = generate_model_input_single(file=input_image)
        return input_tensor

    @staticmethod
    def _post_process(predictions: tf.Tensor) -> int:
        # apply softmax and get our final predictions
        score = tf.nn.softmax(predictions[0])

        results_dict: dict = {}

        for class_id in range(0, 9):
            class_score = score[class_id].numpy()
            # print(f"{class_id} / {class_score}")
            results_dict[class_id] = [class_score]

        results_df: pd.DataFrame = pd.DataFrame(results_dict)

        prediction_class_id: int = results_df.idxmax(axis=1)[0]
        # print(prediction_class_id)
        return prediction_class_id

    def _predict(self, input: tf.Tensor) -> tf.Tensor:
        # Get predictions
        mode_predictions_raw: tf.Tensor = self.model.predict(input, verbose=0)
        return mode_predictions_raw

    def predict(self, input: Path) -> int:
        preprocessed: tf.Tensor = Predictor._pre_process(input_image=input)
        raw_predictions: tf.Tensor = self._predict(input=preprocessed)
        return Predictor._post_process(predictions=raw_predictions)

In [ ]:
predictor = Predictor(model=model)

In [ ]:
# # get input data
# input_image: Path = Path("..") / "data" / "etl" / "test" / "0" / "mnist_test_0_28x28_3.png"
# result = predictor.predict(input=input_image)
# print(result)

In [ ]:
from pathlib import Path

for ground_truth_value in range(0, 9):
    input_path: Path = Path("..") / "data" / "etl" / "test" / str(ground_truth_value)
    for file_path in input_path.glob("*.png"):
        result = predictor.predict(input=file_path)
        print(f"file: {file_path.name}, ground truth: {ground_truth_value}, prediction: {result}")